In [ ]:

# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 

# loading variables from .env file
load_dotenv() 

from classes import Neo4jConnection
from rdflib import Dataset
from rdflib.namespace import RDF

print('Loading data to url: ' + os.getenv('NEO4J_URI'))


In [ ]:
conn = Neo4jConnection.with_user(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), pwd=os.getenv('NEO4J_PASSWORD'))


try:
    numberOfStudents = 1000

    # Clean the database
    res = conn.query(f"match (a) -[r] -> () delete a, r")
    res = conn.query(f"match (a) delete a")
    res = conn.query(f"DROP INDEX entities IF EXISTS")
    res = conn.query(f"DROP INDEX entity_index IF EXISTS")

finally:
    conn.close()       

In [ ]:
conn = Neo4jConnection.with_user(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), pwd=os.getenv('NEO4J_PASSWORD'))


try:
    dataset = Dataset()
    dataset.parse("data/census.trig", "format=trig")

    school_query = """
    PREFIX lpgvs: <https://localhost:4200/lpg/graph_801104/0/rdfs#>
    PREFIX lpgs: <https://localhost:4200/lpg/rdfs#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?schoolCode ?schoolLabel ?schoolPopulation ?zoneCode ?zoneLabel ?areaCode ?areaLabel ?leveeCode ?leveeLabel ?channelCode ?channelLabel
    WHERE {
        ?school a lpgvs:School .
        ?school lpgs:GeoObject-code ?schoolCode .
        ?school rdfs:label ?schoolLabel .   
        ?school lpgvs:School-population ?schoolPopulation .  
        ?zone lpgvs:HasSchoolZone ?school .
        ?area lpgvs:HasFloodRisk ?school .    
        ?levee lpgvs:HasFloodZone ?area .
        ?channel lpgvs:ChannelHasLevee ?levee .         
        ?zone lpgs:GeoObject-code ?zoneCode .
        ?zone rdfs:label ?zoneLabel .      
        ?area lpgs:GeoObject-code ?areaCode .
        ?area rdfs:label ?areaLabel .   
        ?levee lpgs:GeoObject-code ?leveeCode .
        ?levee rdfs:label ?leveeLabel .           
        ?channel lpgs:GeoObject-code ?channelCode .
        ?channel rdfs:label ?channelLabel .            
    }
    """

    for c in dataset.graphs():  # doctest: +SKIP
        if 'graph_801104' in str(c.identifier):
        
            qres = c.query(school_query)
    
            for row in qres:
                
                schoolLabel = row.schoolLabel.replace("'", "\\'")
                zoneLabel = row.zoneLabel.replace("'", "\\'")
                areaLabel = row.areaLabel.replace("'", "\\'")
                leveeLabel = row.leveeLabel.replace("'", "\\'")
                channelLabel = row.channelLabel.replace("'", "\\'")
            
                statement = (
                    f"MERGE (area:Entity:FloodArea {{name: '{areaLabel}', code: '{row.areaCode}'}})"
                    f"MERGE (school:Entity:School {{name: '{schoolLabel}', code: '{row.schoolCode}', numberOfStudents: {int(row.schoolPopulation)}}})"
                    f"MERGE (zone:Entity:SchoolZone {{name: '{zoneLabel}', code: '{row.zoneCode}'}})"                
                    f"MERGE (levee:Entity:Levee {{name: '{leveeLabel}', code: '{row.leveeCode}'}})"                                    
                    f"MERGE (channel:Entity:Channel {{name: '{channelLabel}', code: '{row.channelCode}'}})"                                                        
                    f"MERGE (zone)-[:HAS_SCHOOL]->(school)"
                    f"MERGE (school)-[:HAS_FLOOD_AREA]->(area)"                
                    f"MERGE (area)-[:PROTECTED_BY]->(levee)"                                    
                    f"MERGE (levee)-[:PROTECTS_AGAINST]->(channel)"                                                        
                )
            
                res = conn.query(statement)
finally:
    conn.close()       

In [ ]:
conn = Neo4jConnection.with_user(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), pwd=os.getenv('NEO4J_PASSWORD'))

try:
    dataset = Dataset()
    dataset.parse("data/census.trig", "format=trig")

    school_query = """
    PREFIX lpgvs: <https://localhost:4200/lpg/graph_801104/0/rdfs#>
    PREFIX lpgv: <https://localhost:4200/lpg/graph_801104/0#>
    PREFIX lpgs: <https://localhost:4200/lpg/rdfs#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?areaCode ?areaLabel ?tractCode ?tractLabel ?tractPopulation
    WHERE {
        ?tract a lpgvs:CensusTract .
        ?tract lpgs:GeoObject-code ?tractCode .
        ?tract rdfs:label ?tractLabel .   
        ?tract lpgvs:CensusTract-population ?tractPopulation .
        ?tract lpgvs:TractAtRisk ?area .
        ?area lpgs:GeoObject-code ?areaCode .
        ?area rdfs:label ?areaLabel .           
    }
    """

    for c in dataset.graphs():  # doctest: +SKIP
        if 'graph_801104' in str(c.identifier):
            
            qres = c.query(school_query)
            
            for row in qres:

                areaLabel = row.areaLabel.replace("'", "\\'")
                tractLabel = row.tractLabel.replace("'", "\\'")
            
                statement = (
                    f"MERGE (area:Entity:FloodArea {{name: '{areaLabel}', code: '{row.areaCode}'}})"
                    f"MERGE (tract:Entity:CensusTract {{name: '{tractLabel}', code: '{row.tractCode}', population: {int(row.tractPopulation)}}})"                                                                            
                    f"MERGE (tract)-[:IMPACTS_TRACT]->(area)"                                    
                )
            
                res = conn.query(statement)
finally:
    conn.close()       

In [ ]:
conn = Neo4jConnection.with_user(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), pwd=os.getenv('NEO4J_PASSWORD'))

try:
    conn.query("CREATE FULLTEXT INDEX entities FOR (n:Entity) ON EACH [n.name]")
finally:
    conn.close()       